In [1]:
pip install spyne

Note: you may need to restart the kernel to use updated packages.


In [1]:
from spyne import Application, rpc, ServiceBase, Integer, Unicode , Float
from spyne import Iterable
from spyne.protocol.soap import Soap11
from spyne.server.wsgi import WsgiApplication
from spyne.util.wsgi_wrapper import run_twisted
from spyne.service import ServiceBase
from spyne.model.primitive import String
from spyne.model.complex import ComplexModel
import sys
import logging
from spyne.util.wsgi_wrapper import WsgiMounter
from statistics import mean
import json
import os

In [2]:
class HelloWorldService(ServiceBase):
    @rpc(Unicode, Integer, _returns=Iterable(Unicode))
    def say_hello(ctx, name, times):
        for i in range(times):
            yield 'Hello, %s' % name

In [3]:
class LoanService(ServiceBase):
    @rpc(Unicode, Unicode, _returns=Unicode)  # Accept two Unicode parameters, one for file content and one for file path
    def process_txt_file(ctx, file_content, file_path):
        try:
            # Read the content of the text file
            with open(file_path, 'r') as file:
                file_content = file.read()

            # Process the file content as before
            lines = file_content.split('\n')

            # Initialize variables to store information
            nom = ""
            email = ""
            adresse = ""
            numero = ""
            duree = ""
            description = ""
            montant = 0.00
            adresseMaison = ""
            prixMaison = 0.00

            # Iterate through each line to extract information
            for line in lines:
                if "Nom du client: " in line:
                    nom = line.split("Nom du client: ")[1].strip()
                if "Adresse: " in line:
                    adresse = line.split("Adresse: ")[1].strip()
                if "Email: " in line:
                    email = line.split("Email: ")[1].strip()
                if "Numéro de téléphone: " in line:
                    numero = line.split("Numéro de téléphone: ")[1].strip()
                if "Durée du prêt: " in line:
                    duree = line.split("Durée du prêt: ")[1].split("ans")[0].strip()
                if "Description de la propriété: " in line: 
                    description = line.split("Description de la propriété: ")[1].strip()
                if "Montant du prêt demandé:" in line:
                    montant_str = line.split("Montant du prêt demandé: ")[1].split("EUR")[0].strip()
                    montant = float(montant_str)
                if "Adresse maison:" in line:
                    adresseMaison = line.split("Adresse maison: ")[1].strip()
                if "Prix maison:" in line:
                    prixMaison_str = line.split("Prix maison: ")[1].split("EUR")[0].strip()
                    prixMaison = int(prixMaison_str)
                    
            
            # Create a JSON object with the extracted information
            data = {
                "nom" : nom,
                "email" : email,
                "adresse" : adresse,
                "numero" : numero,
                "duree" : duree,
                "description" : description,
                "montant" : montant,
                "adresseMaison" : adresseMaison,
                "prixMaison" : prixMaison
            }

            # Convert the JSON object to a string
            result = json.dumps(data)

            # Determine the directory of the input file
            #input_directory = os.path.dirname(file_path)

            # Construct the output file path in the same directory
            #output_file_path = os.path.join(input_directory, "output.json")

            # Write the JSON result to the output file
            #with open(output_file_path, 'w') as output_file:
            #    output_file.write(result)            
            
            return result
        except Exception as e:
            return "An error occurred while processing the file."


In [4]:
class ScoreLoanService(ServiceBase):
    @rpc(Integer, Integer, Integer, _returns=Integer)
    def calculate_loan_score(ctx, dettes_en_cours, paiement_en_retard, antecedent_faillite):
        # Logique pour calculer le score en fonction de l'historique financier
        # Ceci est une logique de démonstration, veuillez ajuster selon vos besoins
        score = 100  # Score initial

        if dettes_en_cours > 5000:
            score -= 10

        if paiement_en_retard > 0:
            score -= 20

        if antecedent_faillite:
            score -= 50

        return score

In [5]:
class AnalyseLoanService(ServiceBase):
    @rpc(Float, Float, _returns=Unicode)
    def evaluer_capacite_remboursement(ctx,revenus, depenses):
        # Calculer le solde mensuel (revenus - dépenses)
        solde_mensuel = revenus - depenses
    
        # Vérifier la capacité de remboursement en fonction du solde
        if solde_mensuel > 0:
            capacite_remboursement = "Le client a une capacité de remboursement positive."
        elif solde_mensuel == 0:
            capacite_remboursement = "Le client a juste assez de revenus pour couvrir ses dépenses."
        else:
            capacite_remboursement = "Le client a un solde négatif. Il peut avoir des difficultés à rembourser ses dettes."
    
        return capacite_remboursement

In [6]:
class PropConformService(ServiceBase):
    @rpc(bool, bool, bool, _returns=bool)
    def verifConformHouse(ctx, land_dispute, complies_with_regulations, eligible_for_loan):
        conform = False 
        try:
            if (land_dispute == False) and (complies_with_regulations == True) and (eligible_for_loan == True):
                conform = True
            else :
                conform = False
            return conform
        except Exception as e:
            return conform

In [7]:
class HouseValEstimService(ServiceBase):
    @rpc(Iterable(Float), _returns=Float)
    def valHouseEstim(ctx, input_list):
        # Vérifier que la liste n'est pas vide
        if not input_list:
            raise ValueError("Pas de biens similaires trouvés.")

        # Exemple simple : Somme des valeurs de la liste
        valEstimee = mean(input_list)

        # Vous pouvez ajouter une logique plus complexe ici en fonction de vos besoins

        return valEstimee

In [8]:
# Ca marche ici!

app_houseValEstim = Application([HouseValEstimService], 'valHouseEstim',
                          in_protocol=Soap11(validator='lxml'),
                          out_protocol=Soap11())

app_propConform = Application([PropConformService], 'verifConformHouse',
                          in_protocol=Soap11(validator='lxml'),
                          out_protocol=Soap11())

app_analyse = Application([AnalyseLoanService], 'analyse_service',
                          in_protocol=Soap11(validator='lxml'),
                          out_protocol=Soap11())

app_score = Application([ScoreLoanService], 'score_service',
                          in_protocol=Soap11(validator='lxml'),
                          out_protocol=Soap11())

app_loan = Application([LoanService], 'loan_service',
                          in_protocol=Soap11(validator='lxml'),
                          out_protocol=Soap11())

app_hello = Application([HelloWorldService], 'hello_service',
                          in_protocol=Soap11(validator='lxml'),
                          out_protocol=Soap11())

wsgi_app_houseValEstim = WsgiApplication(app_houseValEstim)
wsgi_app_propConform = WsgiApplication(app_propConform)
wsgi_app_analyse = WsgiApplication(app_analyse)
wsgi_app_score = WsgiApplication(app_score)
wsgi_app_loan = WsgiApplication(app_loan)
wsgi_app_hello = WsgiApplication(app_hello)

twisted_apps = [
    (wsgi_app_houseValEstim, b'valHouseEstim'),
    (wsgi_app_propConform, b'verifConformHouse'),
    (wsgi_app_analyse, b'analyseDemandeService'),
    (wsgi_app_score, b'calculScoreService'),
    (wsgi_app_loan, b'fetchDemandService'),
    (wsgi_app_hello, b'helloWorldService')
]
sys.exit(run_twisted(twisted_apps, 8000))


SystemExit: 

C:\Users\pault\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
